# Analyze Cycle 7 reports for County Executive candidates currently in office

In [1]:
import pandas as pd

### Load analysis from previous reports

In [2]:
pastleaders = pd.read_csv('output/wein-inna-lamb-top.csv')

### Load Cycle 7 reports for 2022

In [3]:
# wein

In [4]:
# inna

In [5]:
# lamb

### Compare Cycle 7 to previous reports

In [6]:
# 